# import necessary packages

In [1]:
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import os
import numpy as np
from pandas import DataFrame


# features of each protein is located in 3 different sources that must be joined together


In [ ]:
physical_features_directory = os.listdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\Ligand+Receptor")
os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\Ligand+Receptor")
physical_features=[]
physicalname=[]
for file in range(0,len(physical_features_directory)) :
    os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\Ligand+Receptor")
    physicalname.append(physical_features_directory[file])
    dfpsaia=pd.read_csv(physical_features_directory[file])
   # print(dfpsaia)
    physical_features.append(dfpsaia)
print(len(physical_features))


In [ ]:
chemical_features_directory = os.listdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\chemical features")
os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\chemical features")
chemicalname=[]
chemical_features=[]
for file in range(0,len(chemical_features_directory)) :
    os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\chemical features")
    chemicalname.append(chemical_features_directory[file])
    chemical=pd.read_csv(chemical_features_directory[file])
    chemical=chemical.drop(['Unnamed: 0'],axis=1)
    #print(chemical)
    chemical_features.append(chemical)
print(len(chemical_features))

In [ ]:
pssm_directory = os.listdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\PSSM")
os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\PSSM")
pssm=[]
for file in range(0,len(pssm_directory)) :
    os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\PSSM")
    pssmfile=pd.read_csv(chemical_features_directory[file])
    pssmfile=pssmfile.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 1', 'Unnamed: 42','Unnamed: 43'],axis=1)
    
    print(pssmfile)
    pssm.append(pssmfile)
print(len(pssm))

# merging protein features from each directory

In [ ]:
def mergeallfeatures(physical_features,chemical_features,pssm):
    all_features=[]
    letterabbreviation= {'CYS': 'C', 'ASP': 'D', 'GLN': 'Q', 'ILE': 'I',
                     'ALA': 'A', 'TYR': 'Y', 'TRP': 'W', 'HIS': 'H',
                     'LEU': 'L', 'ARG': 'R', 'VAL': 'V', 'GLU': 'E',
                     'PHE': 'F', 'GLY': 'G', 'MET': 'M', 'ASN': 'N',
                     'PRO': 'P', 'SER': 'S', 'LYS': 'K', 'THR': 'T',
                     # extended set of amino acids:
                     'MSE': 'M', 'CSE': 'U', 'LNT': 'X', 'GLH': 'E',
                     'HID': 'H', 'HIE': 'H', 'HIP': 'H', 'HYP': 'P'
                    }
    for i in range(len(pssm)):
        f=pd.merge(physical_features[i],chemical_features[i],right_index=True,left_index=True)
        features=pd.merge(f,pssm[i],right_index=True,left_index=True)
        #removing unnecessay columns
        features=features.drop(['ch total ASA|', 'ch b-bone ASA|', 'ch s-chain ASA|','ch polar ASA|','ch n-polar ASA|'],axis=1)
        letters = list(features['res name|'])
        features=features.drop(['res name|'],axis=1)
        letters = [letterabbreviation[base] for base in letters]
        v=''.join(letters)
        v=pd.DataFrame(letters,columns=['res name|'])
        feat=pd.merge(v,features,right_index=True,left_index=True)
        all_features.append(feat)
        print(feat)
    return all_features

In [ ]:
all_features=mergeallfeatures(physical_features,chemical_features,pssm)

In [ ]:
# simulating the interaction process between each cinsecutive two proteins
receptor_ligand=[]
for i in range(0,len(all_features),2):
    ligand=all_features[i]
    receptor=all_features[i+1]
    ligandlenght=len(ligand)
    r=receptor.loc[receptor.index.repeat(ligandlenght)].reset_index(drop=True)
    receptorlenght=len(receptor)
    l= pd.concat([ligand]*receptorlenght, ignore_index=True)
    df=pd.merge(r,l,left_index=True,right_index=True)
    print(df)
    receptor_ligand.append(df)
    

In [ ]:
print(len(receptor_ligand))

In [ ]:
import pandas as pd
import os
import numpy as np
label_directory = os.listdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\negative and positive samples")
os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\negative and positive samples")
label=[]
labelname=[]
for file in range(0,len(label_directory)) :
    os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\negative and positive samples")
    samples=pd.read_csv(label_directory[file])
    label.append(samples)
    labelname.append(label_directory[file])


# each protien has a structure before interaction (where features are extracted) that is different that it's structure after interaction (where label or target is extracted using global alignment we were able to find the common strucutre between the two

In [ ]:
def GET_SCORE(n1, n2, penalty = -1, reward = 1):
     
    if n1 == n2:
        return reward
    else:
        return penalty
 
 
def global_alignment(X, Y, penalty = -1, reward = 1):
     
    # initialize score matrix
    score_matrix = np.ndarray((len(X) + 1, len(Y) + 1))
      
    for i in range(len(X) + 1):
        score_matrix[i, 0] = penalty * i
     
    for j in range(len(Y) + 1):
        score_matrix[0, j] = penalty * j
         
     
    # define each cell in the matrix by as the max score possible in that stage
    for i in range(1, len(X) + 1):
        for j in range(1, len(Y) + 1):
            match = score_matrix[i - 1, j - 1] + GET_SCORE(X[i - 1], Y[j - 1], penalty, reward)
            delete = score_matrix[i -1, j] + penalty
            insert = score_matrix[i, j - 1] + penalty
             
            score_matrix[i, j] = max([match, delete, insert])
             
     
    i = len(X)
    j = len(Y)
     
    global_alignment.align_X = ""
    global_alignment.align_Y = ""
     
    while i > 0 or j > 0:
         
        current_score = score_matrix[i, j]
        left_score = score_matrix[i - 1, j]
         
         
        if i > 0 and j > 0 and X[i - 1] == Y[j - 1]:
            global_alignment.align_X = X[i - 1] + global_alignment.align_X
            global_alignment.align_Y = Y[j - 1] + global_alignment.align_Y
            i = i - 1
            j = j - 1
         
        elif i > 0 and current_score == left_score + penalty:
            global_alignment.align_X = X[i - 1] + global_alignment.align_X
            global_alignment.align_Y = '*' + global_alignment.align_Y
            i = i - 1
             
        else:
            global_alignment.align_X = '*' + global_alignment.align_X
            global_alignment.align_Y = Y[j - 1] + global_alignment.align_Y
            j = j - 1
 
 
    return global_alignment.align_X, global_alignment.align_Y

In [ ]:
def disorder_removing(label,receptor_ligand):
    disorderremoved=[]
    for i in range(0,len(label)):
        print(labelname[i])

        feature=receptor_ligand[i]
        sample=label[i]
        df1=feature
        z=df1['res id|_y']
        f=1
        for i in range(len(z)-1):
            if z[i]<z[i+1]:
                f+=1
        
            else:
                break
        s=df1['res name|_y'][:f]
        a = list(s)
        x = ''.join(str(e) for e in a)
    #print(x)
        df=sample
        d=df['structResIdL']
        q=1
        for i in range(len(d)-1):
            if d[i]<d[i+1]:
                q+=1
        
            else:
                break
        m=df['resNameL'][:q]
        a = list(m)
        y = ''.join(str(e) for e in a)
    #print(y)
        global_alignment(x, y)
        ax=global_alignment.align_X
        ay=global_alignment.align_Y
        print(ax)
        print(ay)
        colum=" ".join(ax)
        colum=colum.split()
        colum1=" ".join(ay)
        colum1=colum1.split()
        dfax = DataFrame (colum,columns=['alignmx'])
        d1=df1['res id|_x']
        c=1
        for d in range(len(d1)-1):
    
            if d1[d]!=d1[d+1]:
                c+=1
        df_repeated = pd.concat([dfax]*c, ignore_index=True)
    #print(df_repeated)
        v=df_repeated.index[df_repeated['alignmx'] == '*'].tolist()
        h=0
        dfa=np.array(df1)
        cols=df1.columns

        l=[]
        for i in v:
            if i<=len(dfa):
            
                dfa=np.insert(dfa,i,"nan",axis = 0)
  
        if v:
            
            lu=pd.DataFrame(dfa,columns=cols)
        else :
            lu=pd.DataFrame(df1,columns=cols)
        lu['alignmx']=df_repeated
        #lu=pd.DataFrame(df_repeated,columns=cols)
        
        dfay1 = DataFrame(colum1,columns=['alignmy'])
        df_repeated = pd.concat([dfay1]*c, ignore_index=True)
        lu["aligny"]=df_repeated
        print(lu)
        lu['Match'] = np.where(lu['aligny'] == "*", 'True', 'False') 
        index_names = lu[lu['Match'] =="True"].index
        lu.drop(index_names, inplace = True)
        lu['Match'] = np.where(lu == "nan", 'True', 'False') 
        index_names = lu[lu['Match'] =="True"].index
        lu.drop(index_names, inplace = True)

        df_repeated = pd.concat([dfay1]*c, ignore_index=True)
    #print(df_repeated)
        v=df_repeated.index[df_repeated['alignmy'] == '*'].tolist()
        w=0
        dfa=np.array(df)
        for i in v:
            if i<=len(dfa):
                dfa=np.insert(dfa,i,"nan",axis = 0)
    #x=np.insert(dfa,l,'nan',axis=0)

        cols=df.columns
        if v:
        
            lb=pd.DataFrame(dfa,columns=cols)
        else:
            lb=df
        lb['alignmy']=df_repeated
        lb=lb.dropna(axis=0)
        df_repeated = pd.concat([dfax]*c, ignore_index=True)
        lb['alignmx']=df_repeated
        lb['Match'] = np.where(lb['alignmx'] == "*", 'True', 'False') 
        index_names = lb[lb['Match'] =="True"].index
        lb.drop(index_names, inplace = True)
        lb=lb.dropna(axis=0)
        index_names = lb[lb['Match'] =="True"].index
        lb.drop(index_names, inplace = True)
        lb['Match'] = np.where(lb == 'nan', 'True', 'False')
        index_names = lb[lb['Match'] =="True"].index
        lb.drop(index_names, inplace = True)
        df3 = pd.merge(lu, lb, left_index=True, right_index=True)
    #print(df3)
        df=lu
        df1=lb
        m=df['res name|_x']
        a = list(m)
        x = ''.join(str(e) for e in a)

        g=df1['resNameR']
        a = list(g)
        y = ''.join(str(e) for e in a)
        df1.reset_index(inplace = True)

        d1=df1['structResIdR']
        li=[]
        for i in range(len(d1)-1):
    
             if d1[i]!=d1[i+1]:
                li.append(a[i])
        li.append(a[-1])
        ah= ''.join(str(e) for e in li)
        df.reset_index(inplace = True)
        d1=df['res id|_x']
        li=[]
        for i in range(len(d1)-1):
    
             if d1[i]!=d1[i+1]:
                li.append(x[i])
        li.append(x[-1])
        x = ''.join(str(e) for e in li)
        global_alignment(x,ah)
        ax=global_alignment.align_X
        ay=global_alignment.align_Y
        print(ax)
        print(ay)
        colum=" ".join(ax)
        colum=colum.split()
        colum1=" ".join(ay)
        colum1=colum1.split()
        dfax = DataFrame (colum,columns=['alignmx'])
        alignmx=dfax.loc[dfax.index.repeat(f)].reset_index(drop=True)
        v=alignmx.index[alignmx['alignmx'] == '*'].tolist()
        h=0
        dfa=np.array(df1)
        l=[]

    #print(v)
        for i in v:
            if i<=len(dfa):
                dfa=np.insert(dfa,i,"nan",axis = 0)
    #x=np.insert(dfa,l,'nan',axis=0)
      
   # x=np.insert(dfa,v,'nan',axis=0)
        cols=df1.columns
        if v:
            ru=pd.DataFrame(dfa,columns=cols)
        else:
            ru=df1
        dfay1 = DataFrame(colum1,columns=['alignmy'])
        alignmy=dfay1.loc[dfay1.index.repeat(f)].reset_index(drop=True)
        print(alignmy)
        v=alignmx.index[alignmx['alignmx'] == '*'].tolist()
        h=0
        dfa=np.array(df)

        for i in v:
            if i<=len(dfa):
            
                dfa=np.insert(dfa,i,"nan",axis = 0)
        
    
    #x=np.insert(dfa,v,'nan',axis=0)
        cols=df.columns
        if v:
            ru=pd.DataFrame(dfa,columns=cols)
        else :
            ru=df
        ru['alignmx']=alignmx
        ru["alignmy"]=alignmy
        ru['Match'] = np.where(ru['alignmy'] == "*", 'True', 'False') 
        index_names = ru[ru['Match'] =="True"].index
        ru.drop(index_names, inplace = True)
        ru['Match'] = np.where(ru == "nan", 'True', 'False') 
        index_names = ru[ru['Match'] =="True"].index
        ru.drop(index_names, inplace = True)
        v=alignmy.index[alignmy['alignmy'] == '*'].tolist()
        h=0
        dfa=np.array(df1)

        for i in v:
            if i<=len(dfa):
            
                dfa=np.insert(dfa,i,"nan",axis = 0)
            
    #x=np.insert(dfa,l,'nan',axis=0)

    #x=np.insert(dfa,v,'nan',axis=0)
        cols=df1.columns
        if v:
            rb=pd.DataFrame(dfa,columns=cols)
        else:
            rb=df1
        rb['alignmy']=alignmy
        rb['alignmx']=alignmx
        rb['Match'] = np.where(rb['alignmx'] == "*", 'True', 'False') 
        index_names = rb[rb['Match'] =="True"].index
        rb.drop(index_names, inplace = True)
        rb['Match'] = np.where(rb== 'nan', 'True', 'False') 
        index_names = rb[rb['Match'] =="True"].index
        rb.drop(index_names, inplace = True)
        ru.reset_index(inplace=True)
        rb.reset_index(inplace=True)
        df3 = pd.merge(ru, rb, left_index=True, right_index=True)
        df3.dropna(axis=0,inplace=True)

        disorderremoved.append(df3)
    return disorderremoved

In [ ]:
removed=disorder_removing(label,receptor_ligand)

In [ ]:
#os.chdir(r'C:\Users\ahmed hatem\Downloads\GP\disorder removed')
#for i in range(len(disorderremoved)):
    #os.chdir(r'C:\Users\ahmed hatem\Downloads\GP\disorder removed')

    #disorderremoved[i].to_csv(labelname[i])

In [ ]:
readyforundersampling=[]
for i in range(0,len(temp)):
    
    protein=temp[i]
    
    protein.dropna(axis=0,inplace=True)

    
    protein=protein.drop(['index_x', 'level_0_x','level_0_y','res id|_x','res id|_y','chain id|_x','chain id|_y','alignmx_x', 'aligny', 'Match_x',
       'alignmy_x',  'chainIdL', 'structResIdL',
       'resNameL', 'chainIdR', 'structResIdR', 'resNameR','alignmx_y', 'Match_y','alignmy_y'],axis=1)
    print(protein)
    readyforundersampling.append(protein)
readyforundersampling

In [ ]:
len(readyforundersampling)

In [ ]:
readyforundersampling=[]
di=os.listdir(r'C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\disorder removed')
os.chdir(r'C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\disorder removed')
for i in range(len(di)):
    os.chdir(r'C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\disorder removed')

    df=pd.read_csv(di[i])
    df=df.drop(['index_x', 'level_0_x','level_0_y','res id|_x','res id|_y','chain id|_x','chain id|_y','alignmx_x', 'aligny', 'Match_x',
       'alignmy_x',  'chainIdL', 'structResIdL',
       'resNameL', 'chainIdR', 'structResIdR', 'resNameR','alignmx_y', 'Match_y','alignmy_y','Unnamed: 0'],axis=1)

    readyforundersampling.append(df)

In [ ]:
readyforundersampling

In [ ]:
def undersampling(readyforundersampling):
    undersampleddata=[]
    for a in range(len(readyforundersampling)):
        #print(readyforundersampling)
        protein=readyforundersampling[a]        
        undersample = RandomUnderSampler()
       # y_under=protein['prediction']
        x = protein.iloc[:, 0 :170]
        y = protein.iloc[:, [171]]
        print(y)
        z=y['prediction']
        for v in range(len(z)):
            if z[v]<0.6:
                z[v]=0
            else:
                z[v]=1
        z=z.astype('float')
        x_under, y_under = undersample.fit_resample(x, z)
    
        #encoding={'A':10000000000000000000,'L':0o01000000000000000000,'I':0o00100000000000000000,'V':0o00010000000000000000,'G':0o00001000000000000000,'K':0o00000100000000000000,'R':0o00000010000000000000,'D':0o00000001000000000000,'E':0o00000000100000000000,'H':0o00000000010000000000,'N':0o00000000001000000000,'Q':0o00000000000100000000,'S':0o00000000000010000000,'T':0o00000000000001000000,'C':0o00000000000000100000,'M':0o00000000000000010000,'Y':0o00000000000000001000,'W':0o00000000000000000100,'F':0o00000000000000000010,'P':0o00000000000000000001}

        #x_under['res name|_x'] = [encoding[base] for base in x_under['res name|_x']]
        #x_under['res name|_y'] = [encoding[base] for base in x_under['res name|_y']]
        #x_under=pd.DataFrame(x_under)
        #y_under=pd.DataFrame(y_under)
        undersampled=pd.merge(x_under,y_under,left_index=True,right_index=True)
        undersampl=shuffle(undersampled)
        #undersampl=undersampl.reset_index(inplace=True)

        #print(undersampl )

        undersampleddata.append(undersampl)
    return undersampleddata

In [ ]:
undersampleddata=undersampling(readyforundersampling)

In [ ]:
print(undersampleddata)

In [ ]:
os.chdir(r'C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\undersampled data_no-encoding')
for i in range(len(undersampleddata)):
    os.chdir(r'C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\undersampled data_no-encoding')

    undersampleddata[i].to_csv(labelname[i])

In [ ]:
print(len(undersampleddata))

In [ ]:
pprint(undersampleddata)

In [ ]:
print(undersampleddata)